<a href="https://colab.research.google.com/github/owen-shaffer/DataScience-Project/blob/main/DSSProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/owen-shaffer/DataScience-Project

Cloning into 'DataScience-Project'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 50 (delta 18), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 2.44 MiB | 2.39 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [16]:
import json
from pandas import json_normalize
import csv
import pandas as pd
import sqlite3

#Functions
#These functions mostly take in two parameters, the data file and a name for the converted data file. The function opens the file and reads it into a
#pandas dataframe. From there the data can be saved into the desired format.
#When converting into an sql database table, the name of the new table is also a parameter

def convert_csv_to_json(csv_file, json_file):
    with open(csv_file, 'r') as s:
      df = pd.read_csv(s)
      df.to_json(json_file, orient='records')

def convert_json_to_csv(json_file, csv_file):
  with open(data, 'r') as s:
    json_data = json.load(s)
    df = pd.json_normalize(json_data) # Within json data sometimes there are lists within dictionaries or vice versa. Pandas does not like turning this into a df so it needs to be 'flattened'. The normalize function converts this nested data into rows and columns
    df.to_csv(csv_file)

def convert_csv_to_sql(csv_file, sql_file, table):
    with open(csv_file, 'r') as s:
      df = pd.read_csv(s)
      df.to_sql(table, con=sqlite3.connect(sql_file))#Writes data to a table in a sql database

def convert_json_to_sql(json_file, sql_file,table):
    with open(json_file, 'r') as s:
      json_data = json.load(s)
      df = pd.json_normalize(json_data)
      for col in df.columns: #This code accounts for lists and dictioneries in the json data by turning them into strings. Without it the data cant be converted into a .db because sql only accepts basic text, int, float, null.
      #The code iterates over each column and checks its type if it is an object so a list or dictionary then it goes through the conversion.
        if df[col].dtype == 'object':
            df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, (dict, list)) else x)
      df.to_sql(table, con=sqlite3.connect(sql_file), index=False)

def convert_sql(sql_file, table, new_file):
  conn = sqlite3.connect(sql_file) #Connects to the database
  df = pd.read_sql("SELECT * FROM"+table, conn) #Gets the data from given table in the database
  input = input("Would you like to convert your data to csv (1) or json(2)")
  if input == '1':
    df.to_csv(new_file)
  elif input == '2':
    df.to_json(new_file)
  else:
    print("Invalid input")

#This function takes in a dataframe and prints the number of rows and columns by taking the values from the .shape function output
def summarize_data(df):
    print("\n Data Summary")
    print(f"Number of rows: {df.shape[0]}")
    print(f"Number of columns: {df.shape[1]}")



#Data ingesting and converting

data = input("Provide the path for your data: ")

#This chunk opens the data based on file type and turns it into a datafram
if '.json' in data:
  with open(data, 'r') as f:
    json_data = json.load(f)
    df = pd.json_normalize(json_data)# Within json data sometimes there are lists within dictionaries or vice versa. Pandas does not like turning this into a df so it needs to be 'flattened'. The normalize function converts this nested data into rows and columns
if '.csv' in data:
  df = pd.read_csv(data)
if '.db' in data:
  conn = sqlite3.connect(data)
  table = input("What is the name of your table? ")
  query = "SELECT * FROM " + table
  data = pd.read_sql(query, conn)
  df = pd.DataFrame(data)

summarize_data(df)

print('')
print('This is the first 5 rows of the data')
print(df.head(), flush=True)

print('')
print('')

remove = input("Would you like to remove any columns? (y/n) ")
if remove == 'y':
  print("What range of columns would you like to remove? ")
  x_columns = input("Beginning at column: ")
  y_columns = input("Ending at column: ")
  if int(x_columns) > int(y_columns): #Range would not work if x was greater than y
    print("Invalid range")
  else:
    df = df.drop(df.columns[int(x_columns):int(y_columns)], axis=1) #Removes columns based on the range given
    print(df.head())
    print('')
    print('')
  if int(x_columns)>len(df.columns): #This accounts for the numbers provided being outside the number of columns
    print("Invalid range")
  else:
    print("Columns removed")
else:
  print("No columns removed")

print('')
print('')
choice = input("Would you like to convert your data to csv (1), json(2), or sql (3)? ")
name = input("What would you like to name your file? ")
print('')

#This section uses the functions previously defined to convert data into different formats given the users inputs
if choice == '1':
  if '.csv' in data:
    print("Your file is already in csv format")
  else:
    if '.json' in data:
      convert_json_to_csv(data, name + '.csv')
    if '.db' in data:
      convert_sql(data, name + '.csv')

if choice == '2':
  if '.json' in data:
    print("Your file is already in json format")
  else:
    if '.db' in data:
      convert_sql(data, name + '.json')
    if '.csv' in data:
      convert_csv_to_json(data, name + '.json')

if choice == '3':
  file_type = input("Is your file a csv (1) or json (2)? ")
  if file_type == '1':
    database_path = input("What is the path to your database? ")
    table = input("What is the name of your table? ")
    convert_csv_to_sql(data, database_path, table)
  if file_type == '2':
    database_path = input("What is the path to your database? ")
    table = input("What is the name of your table? ")
    convert_json_to_sql(data, database_path, table)

summarize_data(df)

Provide the path for your data: /content/plastictest.db
What is the name of your table? plastictest

 Data Summary
Number of rows: 44226
Number of columns: 9

This is the first 5 rows of the data
   index SampleMatrix              SampleID  \
0      0        manta  CB-4-Manta-21Aug2017   
1      1        manta  CB-4-Manta-21Aug2017   
2      2        manta  CB-4-Manta-21Aug2017   
3      3        manta  CB-4-Manta-21Aug2017   
4      4        manta  CB-4-Manta-21Aug2017   

                            PlasticType MorphologicalCategory       Color  \
0  Polyethylene/polypropylene copolymer                 Fiber  Light Blue   
1                             Polyester                 Fiber       Black   
2  Polyethylene/polypropylene copolymer                 Fiber       Clear   
3                             Polyester                 Fiber  Light Blue   
4                             Polyester                 Fiber       Black   

   Length.mm  Width.mm                                Matr